In [1]:
import pandas as pd

FILE = "/Users/Shared/data/HN_posts_year_to_Sep_26_2016.csv"
SIZE = 50000
VIRAL_THRESHOLD = 3

data = pd.read_csv(FILE)
data = data.sample(SIZE)

In [2]:
data = data[['title', 'num_points']]

In [3]:
viral = data["num_points"].apply(lambda x: pd.Series(1 if x >= VIRAL_THRESHOLD else 0))
viral.columns=["viral"]

In [4]:
#skew checking
viral.describe()

,viral
count,50000.000000
mean,0.417400
std,0.493135
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [5]:
import nltk
import numpy as np
title = data["title"].apply(
    lambda x: pd.Series(nltk.word_tokenize(x)))
title = title.applymap(lambda x: '' if x is np.nan else x.lower())

In [6]:
import DataLoader.GloVe as gl
glove = gl.load('/Users/Shared/data/glove.6B/glove.6B.50d.txt')

Start: Loading Glove Model
End: Loaded 400000 rows.


In [7]:
def to_word_vec(word, embedding):
    try:
        return embedding.loc[word]
    except:
        #print(word)
        return embedding.loc['.']

title_emb = []
for i in title.columns:
    print(i)
    title_emb.append(title[i].apply(lambda x: to_word_vec(x, glove)))
    title_emb[-1].columns = [ i * glove.shape[1] + j for j in title_emb[i].columns]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [8]:
hn_data = viral
for i in range(len(title_emb)):
    hn_data = hn_data.join(title_emb[i])

In [12]:
testing_data = hn_data.sample(frac=0.2)
training_data = hn_data.drop(testing_data.index)


testing_data.to_csv('./hn_testing_%d_%d.csv' % (SIZE, VIRAL_THRESHOLD), header=False, index=False)
training_data.to_csv('./hn_training_%d_%d.csv' % (SIZE, VIRAL_THRESHOLD), header=False, index=False)